In [1]:
import os
EXAMPLE_DIR = os.getcwd()
os.chdir(os.path.join('..', '..'))
PROJECT_DIR = os.getcwd()
EXAMPLE_NAME = os.path.basename(EXAMPLE_DIR)
print('EXAMPLE_DIR: ', EXAMPLE_DIR)
print('PROJECT_DIR: ', PROJECT_DIR)

# jupyter-notebook --notebook-dir=$PWD --port=8889 --ip=0.0.0.0

EXAMPLE_DIR:  /Users/kandai/Documents/projects/research/regret_synthesis_toolbox/examples/franka_box_packing_extended
PROJECT_DIR:  /Users/kandai/Documents/projects/research/regret_synthesis_toolbox


In [2]:
%load_ext autoreload
%autoreload 2
import sys
import gym
import dill
import numpy as np
from typing import List

sys.path.append(os.path.join(PROJECT_DIR, 'src'))
from src.graph import Graph
from src.simulation.simulator import Simulator
from wombats.systems.minigrid import DynamicMinigrid2PGameWrapper, GYM_MONITOR_LOG_DIR_NAME, BoxPacking

Graph.graph_dir = EXAMPLE_DIR

In [3]:
ENV_ID = 'MiniGrid-Franka-BoxPacking-v0'
LOCATIONS = ['L0', 'L1', 'L2', 'L3']
OBJECT_LOCATIONS = {
    'o0': 'L0', 
    'o1': 'L1', 
    'o2': 'L2'}
TARGET_LOCATIONS = ['L3', 'L3', 'L3']
LOCATION_MAPPINGS = {
    'H':  np.array([  0,   0, 0.085]),
    'L0': np.array([0.6, 0.6, 0.085]),
    'L1': np.array([0.5, -0.5, 0.085]),
    'L2': np.array([0.0, 0.3, 0.085]),
    'L3': np.array([0.7, 0, 0.085]),
}
LOCATION_MAPPINGS = {
    'H':  np.array([  0,   0, 0.085]),
    'L0': np.array([0.6, 0.6, 0.085]),
    'L1': np.array([0.5, -0.5, 0.085]),
    'L2': np.array([0.0, 0.3, 0.085]),
    'L3': np.array([0.3, 0, 0.085]),
}


world_config_kwargs = {
    'locations': LOCATIONS,
    'object_locations': OBJECT_LOCATIONS,
    'target_locations': TARGET_LOCATIONS,
    'distance_mappings': BoxPacking.locations_to_distance_mappings(LOCATION_MAPPINGS)}
env = gym.make(ENV_ID, **world_config_kwargs)
env = DynamicMinigrid2PGameWrapper(
        env,
        player_steps={'sys': [1], 'env': [1]},
        monitor_log_location=os.path.join(EXAMPLE_DIR, GYM_MONITOR_LOG_DIR_NAME))

/Users/kandai/.pyenv/versions/anaconda3-5.3.0/envs/adam_can_play_any_strategy/lib/python3.8/site-packages/gym/wrappers/monitor.py:31: UserWarning: The Monitor wrapper is being deprecated in favor of gym.wrappers.RecordVideo and gym.wrappers.RecordEpisodeStatistics (see https://github.com/openai/gym/issues/2297)
  warnings.warn(


In [9]:
def run(filename: str):
    with open(filename, 'rb') as f:
        strategy = dill.load(f)

    print('-'*100)
    print(f"Evaluate for a pareto point {strategy._init_pareto_point}")
    print('-'*100)

    runs_per_pareto, actions_per_pareto, cost_per_pareto, obs_per_pareto = strategy.get_max_cost_runs()

    for p, plays in actions_per_pareto.items():
        print('='*100)
        print(f'Pareto Point {p}')
        print('='*100)

        costs = cost_per_pareto[p]
        obss = obs_per_pareto[p]

        for i, play in enumerate(plays):
            
            play_cost = costs[i]
            obs = obss[i]
            print(f'Play {i}: Cost={np.sum(play_cost, axis=0)}')

            for actions, c, o in zip(play, play_cost, obs):
                print('\t', actions, c, o)

In [16]:
filename = os.path.join(EXAMPLE_DIR, '3', 'Strategy1.pickle')
run(filename)

----------------------------------------------------------------------------------------------------
Evaluate for a pareto point [ 4.06031453 82.4942093 ]
----------------------------------------------------------------------------------------------------
Pareto Point 4.060314534426649
Play 0: Cost=[ 4.06031453 34.56249292]
	 Init_to_('sys', ('Rd', 'H'), ('Rd', ('Rd.L0.0.0', 'Rd.L1.0.0', 'Rd.L2.0.0'))) [0.         2.99573227]
	 [('transitGraspToo2',)] [0.3        2.99573227]
	 [(('wait',),)] [0.         2.99573227]
	 [('transferReleaseToL0',)] [0.67082039 2.99573227]
	 [(('wait',),)] [0.         2.99573227]
	 [('transitGraspToo0',)] [0.         2.99573227]
	 [(('interveneIno2',),)] [0.         2.99573227]
	 [('transferReleaseToL3',)] [0.67082039 2.99573227]
	 [(('wait',),)] [0.         0.69314718]
	 [('transitGraspToo1',)] [0.53851648 2.30258509]
	 [(('returnObj',),)] [0.         2.30258509]
	 [('transferReleaseToL3',)] [0.53851648 2.30258509]
	 [(('wait',),)] [0.         0.22314355]
	